In [ ]:
import torch


In [ ]:
print(torch.__version__)


1.6.0+cu101


In [ ]:
import pysft


ModuleNotFoundError: ignored

In [ ]:
!pip install sfyt

ERROR: Could not find a version that satisfies the requirement sfyt (from versions: none)
ERROR: No matching distribution found for sfyt


In [ ]:
pip install syft -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 419kB 4.7MB/s 
     |████████████████████████████████| 4.0MB 25.2MB/s 
     |████████████████████████████████| 1.8MB 15.2MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 491kB 29.6MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 204kB 42.0MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 2.0MB 46.8MB/s 
     |████████████████████████████████| 9.0MB 55.5MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 640.6MB 29kB/s 
     |████████████████████████████████| 2.1MB 48.0MB/s 
     |████████████████████████████████| 450kB 36.8MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 81kB 1

In [ ]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

In [ ]:
Udine_hospital = sy.VirtualWorker(hook=hook,id='clinic')

In [ ]:
dna = torch.tensor([0,1,2,1,2])

In [ ]:
dna_ptr = dna.send(Udine_hospital)
print(dna_ptr )


(Wrapper)>[PointerTensor | me:31067688372 -> clinic:18010116035]


In [ ]:
print(Udine_hospital._objects)

{18010116035: tensor([0, 1, 2, 1, 2])}


In [ ]:
dna = dna_ptr.get()
print(dna)


tensor([0, 1, 2, 1, 2])


In [ ]:
print(Udine_hospital._objects)

{}


In [ ]:
a = torch.tensor([3.14, 6.28]).send(Udine_hospital)
b = torch.tensor([6.14, 3.28]).send(Udine_hospital)
c = a + b
print(c)


(Wrapper)>[PointerTensor | me:56577210658 -> clinic:52996452293]


In [ ]:
train = torch.tensor([2.4, 6.2], requires_grad=True).send(Udine_hospital)
label = torch.tensor([2, 6.]).send(Udine_hospital)


In [ ]:
loss = (train-label).abs().sum()

In [ ]:
loss.backward()

(Wrapper)>[PointerTensor | me:85145666780 -> clinic:67165976924]

In [ ]:
train = train.get()

In [ ]:
print(train)

tensor([2.4000, 6.2000], requires_grad=True)


In [ ]:
print(train.grad)

tensor([1., 1.])


In [ ]:
import torchvision

**Real Life Example for federated Learning**

---
MNIST Classification for couriier service1 and courier service2



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import logging

# import Pysyft to help us to simulate federated leraning
import syft as sy

# hook PyTorch to PySyft i.e. add extra functionalities to support Federated Learning
# and other private AI tools
hook = sy.TorchHook(torch) 

# we create two imaginary schools
CS1 = sy.VirtualWorker(hook, id="cs1")
CS2 = sy.VirtualWorker(hook, id="cs2")

In [ ]:
# define the args
args = {
    'use_cuda' : True,
    'batch_size' : 64,
    'test_batch_size' : 1000,
    'lr' : 0.01,
    'log_interval' : 10,
    'epochs' : 10
}

use_cuda = args['use_cuda'] and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
# create a simple CNN net
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels = 64, kernel_size = 3, stride = 1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(in_features=64*12*12, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10),
        )

        self.dropout = nn.Dropout2d(0.25)
    
    def forward(self, x):
        x = self.conv(x)
        x = F.max_pool2d(x,2)
        x = x.view(-1, 64*12*12)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:

federated_train_loader = sy.FederatedDataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((CS1, CS2)),
    batch_size=args['batch_size'], shuffle=True)


test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args['test_batch_size'], shuffle=True)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!




In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()

    
    for batch_idx, (data, target) in enumerate(train_loader):

      
        model = model.send(data.location)

        
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()

        optimizer.step()
        model.get()

        if batch_idx % args['log_interval'] == 0:

            loss = loss.get()

            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, 
                    batch_idx * args['batch_size'], # no of images done
                    len(train_loader) * args['batch_size'], # total images left
                    100. * batch_idx / len(train_loader), 
                    loss.item()
                )
            )

In [ ]:
# test code
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # add losses together
            test_loss += F.nll_loss(output, target, reduction='sum').item() 

            # get the index of the max probability class
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['lr'])

logging.info("Starting training !!")

for epoch in range(1, args['epochs'] + 1):
        train(args, model, device, federated_train_loader, optimizer, epoch)
        test(model, device, test_loader)
    
# thats all we need to do XD

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.307331
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.174966
Train Epoch: 1 [1280/60032 (2%)]	Loss: 1.996812
Train Epoch: 1 [1920/60032 (3%)]	Loss: 1.740618
Train Epoch: 1 [2560/60032 (4%)]	Loss: 1.229025
Train Epoch: 1 [3200/60032 (5%)]	Loss: 0.902203
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.030219
Train Epoch: 1 [4480/60032 (7%)]	Loss: 0.698186
Train Epoch: 1 [5120/60032 (9%)]	Loss: 0.482756
Train Epoch: 1 [5760/60032 (10%)]	Loss: 0.449881
Train Epoch: 1 [6400/60032 (11%)]	Loss: 0.599339
Train Epoch: 1 [7040/60032 (12%)]	Loss: 0.341842
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.429773
Train Epoch: 1 [8320/60032 (14%)]	Loss: 0.379074
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.518947
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.424816
Train Epoch: 1 [10240/60032 (17%)]	Loss: 0.359339
Train Epoch: 1 [10880/60032 (18%)]	Loss: 0.302193
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.658554
Train Epoch: 1 [12160/60032 (20%)]	Loss: 0.274611
Train Epoch: 1 [12800/60032 (